log

local cv: 
 
*     f1 micro : 0.853646
     
*     all_time :  1 min 01 sec
     
*     estimated for hidden test 2k text files :  0 hr 10 min
    
    
actual public lb:

*     f1 micro : 0.848
     
*     all_time :  10 min 

In [ ]:
if 1:
    # https://www.kaggle.com/nbroad/deberta-v2-3-fast-tokenizer
    import shutil
    from pathlib import Path
    
    transformers_path = Path('/opt/conda/lib/python3.7/site-packages/transformers')
    input_dir = Path('../input/nbme-submit-00/deberta_v2_convert_tokenizer')
    
    convert_file = input_dir / 'convert_slow_tokenizer.py'
    conversion_path = transformers_path/convert_file.name
    if conversion_path.exists():
        conversion_path.unlink()
    shutil.copy(convert_file, transformers_path)
    
    deberta_v2_path = transformers_path / 'models' / 'deberta_v2'
    for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
        filepath = deberta_v2_path/filename
        if filepath.exists():
            filepath.unlink()
        shutil.copy(input_dir/filename, filepath)

        
import sys
sys.path.append('../input/nbme-submit-00')

import os
import numpy as np
import glob
import pandas as pd
import itertools
import ast
from sklearn import metrics

from timeit import default_timer as timer
import psutil
import gc

is_amp   = True  #True #False
is_cuda  = True
is_debug = False



In [ ]:
# helper---------------------------------------------------------------------------------
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)
    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)
    else:
        raise NotImplementedError

#https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
def memory_used_to_str():
    pid = os.getpid()
    processs = psutil.Process(pid)
    memory_use = processs.memory_info()[0] / 2. ** 30
    return 'use ram memory gb ' + str(np.round(memory_use, 2))


In [ ]:
max_length = 512
threshold_score = 0.5

In [ ]:
#dataset ######################################################################################
submit_dir = ''
feature_csv_file      = '../input/nbme-score-clinical-patient-notes/features.csv'
patient_note_csv_file = '../input/nbme-score-clinical-patient-notes/patient_notes.csv'

if is_debug: 
    valid_csv_file = '../input/nbme-score-clinical-patient-notes/train.csv'
    valid_df = pd.read_csv(valid_csv_file)
    
    fold_df = pd.read_csv('../input/nbme-submit-00/valid_fold0.id.csv')
    valid_df = valid_df[valid_df.id.isin(fold_df.id)].reset_index(drop=True)

else:
    valid_csv_file = '../input/nbme-score-clinical-patient-notes/test.csv' 
    valid_df = pd.read_csv(valid_csv_file)
    
    
#---
feature_df = pd.read_csv(feature_csv_file)
feature_df.loc[27, 'feature_text'] = 'Last-Pap-smear-1-year-ago'

patient_note_df = pd.read_csv(patient_note_csv_file)
valid_df = valid_df.merge(feature_df, on=['feature_num', 'case_num'], how='left')
valid_df = valid_df.merge(patient_note_df, on=['pn_num', 'case_num'], how='left')

print('CHECK DF FILES ####################################')
print('len(valid_df)',len(valid_df))
print('')
for t in range(3): print(valid_df.iloc[t],'\n')
print('ok!')

#<todo> sort text for speed?



def get_loader(tokenizer, batch_size):
    from torch.utils.data.dataset import Dataset
    from torch.utils.data import DataLoader
    from torch.utils.data.sampler import SequentialSampler
    import torch
    
    def null_collate_fn(batch):
        d = {}
        key = batch[0].keys()
        for k in key:
            v = [b[k] for b in batch]
            if k in ['token_in_mask', 'token_out_mask', 'token_type_id', 'token_id',]:
                v = torch.stack(v)
            d[k] = v
        return d

    class NBMEDataset(Dataset):
        def __init__(self, df, tokenizer, max_length):
            self.df = df
            self.max_length = max_length
            self.tokenizer  = tokenizer
            self.length     = len(self.df)
        
        def __len__(self):
            return self.length
        
        def __getitem__(self, index):
            d    = self.df.iloc[index]
            id   = d['id']
            pn_history   = d.pn_history
            feature_text = d.feature_text
            
            #text to token
            e = tokenizer(
                pn_history,
                feature_text,
                add_special_tokens=True,
                max_length=max_length,
                padding='max_length',
                return_offsets_mapping=True
            )
            token_in_mask  = e['attention_mask']
            token_type_id  = e['token_type_ids']
            token_id       = e['input_ids']
            token_offset   = e['offset_mapping']
            token_out_mask = (np.array(token_in_mask)==1)&(np.array(token_type_id)==0).astype(np.int8).tolist()
            
 
            #-------------------------------------
            r = {}
            r['index'] = index
            r['id'   ] = id
            r['pn_history'    ] = pn_history
            r['feature_text'  ] = feature_text
            r['token_offset'  ] = token_offset
            r['token_out_mask'] = torch.tensor(token_out_mask, dtype=torch.long)
            r['token_in_mask' ] = torch.tensor(token_in_mask , dtype=torch.long)
            r['token_type_id' ] = torch.tensor(token_type_id , dtype=torch.long)
            r['token_id'      ] = torch.tensor(token_id      , dtype=torch.long)
            return r
        
    valid_dataset = NBMEDataset(valid_df, tokenizer, max_length)
    valid_loader  = DataLoader(
        valid_dataset,
        sampler = SequentialSampler(valid_dataset),
        batch_size  = batch_size, #4, #
        drop_last   = False,
        num_workers = 2, #0, #
        pin_memory  = False,
        collate_fn = null_collate_fn,
    )
    return valid_loader




In [ ]:
## post processing #######################################################

def array_to_span(array):
    span = [list(g) for _, g in itertools.groupby(np.where(array==1)[0], key=lambda n, c=itertools.count(): n - next(c))]
    span = ['%d %d'%(min(r), max(r)+1) for r in span]
    #location = ';'.join(span)
    return span


def location_to_array(location, pn_history, format='truth'):
    if format=='truth':
        location = location.replace('"', "'").replace(';', "','")
        location = ast.literal_eval(location)
    if format=='predict':
        if location is not '':
            location = location.replace(';', ',')
            location = location.split(',')
            #print(location)
    
    array = np.zeros(len(pn_history))
    for loc in location:
        #print(loc)
        start, end = loc.split()
        start, end = int(start), int(end)
        array[start:end]=1
    return array


def compute_lb_f1_score(predict_df, truth_df):
    assert(predict_df['id']==truth_df['id']).all()
    L = len(truth_df)

    truth    = []
    predict  = []
    for i in range(L):
        dt = truth_df.iloc[i]
        dp = predict_df.iloc[i]
        t = location_to_array(dt.location, dt.pn_history, format='truth')
        p = location_to_array(dp.location, dt.pn_history, format='predict')
        truth.append(t)
        predict.append(p)
        
    
    predict = np.concatenate(predict)
    truth = np.concatenate(truth)
    lb = metrics.f1_score(truth, predict)

    return lb
    
    

In [ ]:
## post processing #######################################################

def array_to_span(array):
    span = [list(g) for _, g in itertools.groupby(np.where(array==1)[0], key=lambda n, c=itertools.count(): n - next(c))]
    span = ['%d %d'%(min(r), max(r)+1) for r in span]
    #location = ';'.join(span)
    return span


def location_to_array(location, pn_history, format='truth'):
    if format=='truth':
        location = location.replace('"', "'").replace(';', "','")
        location = ast.literal_eval(location)
    if format=='predict':
        if location is not '':
            location = location.replace(';', ',')
            location = location.split(',')
            #print(location)
    
    array = np.zeros(len(pn_history))
    for loc in location:
        #print(loc)
        start, end = loc.split()
        start, end = int(start), int(end)
        array[start:end]=1
    return array


def compute_lb_f1_score(predict_df, truth_df):
    assert(predict_df['id']==truth_df['id']).all()
    L = len(truth_df)

    truth    = []
    predict  = []
    for i in range(L):
        dt = truth_df.iloc[i]
        dp = predict_df.iloc[i]
        t = location_to_array(dt.location, dt.pn_history, format='truth')
        p = location_to_array(dp.location, dt.pn_history, format='predict')
        truth.append(t)
        predict.append(p)
        
    
    predict = np.concatenate(predict)
    truth = np.concatenate(truth)
    lb = metrics.f1_score(truth, predict)

    return lb
    

In [ ]:
# model ##################################################################

ensemble =(
    {
        'arch' : '../input/nbme-submit-00/microsoft-deberta-v3-small',
        'checkpoint' : 	[
            '../input/nbme-submit-00/00004000.model.pth',
            #'/root/share1/kaggle/2022/NBME-patient-notes/result/run02/debert-v3-small-xx1/fold-1/checkpoint/00006000.model.pth',
        ],
        'batch_size' : 64,
    },
    
 

)

def load_model_and_run(config):
    
    if config['arch']=='../input/nbme-submit-00/microsoft-deberta-v3-small':
        from deberta_v3_small_model import Net, get_tokenizer
    # elif config['arch']=='./microsoft-deberta-v3-large':
    # 	from deberta_v3_large_model import Net, get_tokenizer
    else:
        raise NotImplementedError
  
    import torch
    from torch.nn.parallel.data_parallel import data_parallel
    import torch.cuda.amp as amp
    
    #from torch.profiler import profile, record_function, ProfilerActivity
    #with profile(activities=[ProfilerActivity.CPU],
    #           profile_memory=True, record_shapes=True) as prof:
    #--------------------------------------------------------------------------
    
    num_fold = len(config['checkpoint'])
    net = Net(config['arch'])
    net.eval()
    if is_cuda:
        net.cuda()
 
    tokenizer = get_tokenizer(config['arch'])
    valid_loader = get_loader(tokenizer, config['batch_size'])
    
    result = None
    for n in range(num_fold) :
        net.load_state_dict(torch.load(config['checkpoint'][n], map_location=lambda storage, loc: storage)['state_dict'],strict=False)
        print('load "%s" ok'%config['checkpoint'][n])
        
        r = {
            'token_out_mask':[],
            'token_offset':[],
            'probability' :[],
            'correction'  :[],
        }
        
        num_valid=0
        start_timer = timer()
        for t, batch in enumerate(valid_loader):
            batch_size = len(batch['index'])
            if is_cuda:
                batch['token_in_mask'] = batch['token_in_mask'].cuda()
                batch['token_type_id'] = batch['token_type_id'].cuda()
                batch['token_id'     ] = batch['token_id'     ].cuda()
                
            with torch.no_grad():
                with amp.autocast(enabled=is_amp):
                    output = data_parallel(net,batch)
                    
                    r['token_offset'  ] += batch['token_offset']
                    r['token_out_mask'].append( batch['token_out_mask'].data.cpu().numpy() )
                    r['probability'   ].append( output['token_label'].data.cpu().numpy() )
                    r['correction'    ].append( output['token_correction'].data.cpu().numpy() )
                    num_valid += batch_size
                    
            print('\r\t%d/%d  %s'%(num_valid, len(valid_loader.dataset), time_to_str(timer() - start_timer,'sec')),end='',flush=True)
        print('')
        if result is None:
            r['token_offset'   ] = np.array(r['token_offset'], object)
            r['token_out_mask' ] = np.concatenate(r['token_out_mask'])
            r['probability'    ] = np.concatenate(r['probability'])
            r['correction'     ] = np.concatenate(r['correction'])
            result = r
        else:
            result['probability'] += np.concatenate(r['probability'])
            result['correction' ] += np.concatenate(r['correction'])
    #-----------------------------------------------------------------------
    #average
    result['probability'] /= num_fold
    result['correction' ] /= num_fold
    result['correction' ] = result['correction'].astype(np.uint8)
    #print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))
    
    del net, tokenizer
    del valid_loader
    gc.collect()
    if is_cuda: torch.cuda.empty_cache()
    
    return result


def run_submit():
    if is_debug: print("THIS IS DEBUG ####################################")
    all_time = 0
    print('start', memory_used_to_str())
    
    ensemble_result = []
    for config in ensemble:
        start_timer = timer()
        result = load_model_and_run(config)
        ensemble_result.append(result)
        all_time += timer() - start_timer
        print('')
        print('end', memory_used_to_str())
    num_model = len(ensemble_result)
    
    
    #-----------------------------------
    submit_df = []
    for i,d in valid_df.iterrows():
     
        #ensemble -----
        prob_pn_history = np.full((len(d.pn_history)),0, np.float32)
        for m in range(num_model):
            #<todo> verify id
            
            p = ensemble_result[m]['probability'][i]
            c = ensemble_result[m]['correction' ][i]
            token_offset = ensemble_result[m]['token_offset'][i]
            
            T = ensemble_result[m]['token_out_mask'][i].sum()
            for t in range(T):
               start,end = token_offset[t]
               s = start + c[t,0]
               e = end   - c[t,1]
               prob_pn_history[s:e] += p[t]
        prob_pn_history = prob_pn_history/num_model
        #ensemble -----
        
        predict = prob_pn_history>threshold_score
        span = array_to_span(predict)#[:-1]
        location = ';'.join(span)
        submit_df.append({'id':d.id, 'location':location})
        
        #print('\r preparing submit_df :', i, d.id, len(d.pn_history), end ='', flush=True)
    print('\n')

    #----------------------------------------
    submit_df = pd.DataFrame(data = submit_df)
    print(submit_df)
    submit_df.to_csv('submission.csv', index=False)
 
    

    print('------------------')
    for t in range(3): print(submit_df.iloc[t],'\n')
    print('submission ok!----')

    if is_debug:
        print("THIS IS DEBUG ####################################")
        f1_score = compute_lb_f1_score(submit_df, valid_df)
        print('f1 micro : %f\n' % f1_score)
  
        #2000 patient notes in the test set.
        print('all_time : %s'%(time_to_str(all_time,'sec')))
        print('estimated for 10k text files : %s'%(time_to_str(all_time/len(valid_df)*30_000,'min')))


In [ ]:
run_submit()